## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-30-14-54-00 +0000,wsj,President Trump’s tariff rollout has taken num...,https://www.wsj.com/economy/trade/trump-tariff...
1,2025-07-30-14-48-30 +0000,startribune,Twins shortstop Carlos Correa could be on trad...,https://www.startribune.com/twins-shortstop-ca...
2,2025-07-30-14-46-09 +0000,nyt,"Trump Threatens India With Steep Tariffs, Incl...",https://www.nytimes.com/2025/07/30/us/politics...
3,2025-07-30-14-45-43 +0000,nyt,Brazil’s President Lula Voices Frustration Wit...,https://www.nytimes.com/2025/07/30/world/ameri...
4,2025-07-30-14-45-36 +0000,nyt,Tsunami Alerts Are Mostly Lifted After Major P...,https://www.nytimes.com/live/2025/07/29/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2364/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
1,trump,62
37,tsunami,24
185,new,20
149,gaza,19
307,palestinian,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
209,2025-07-29-22-00-59 +0000,nyt,"Trump Says Gaza Will Get More Aid, but Israel ...",https://www.nytimes.com/2025/07/29/us/politics...,126
154,2025-07-30-01-39-32 +0000,nypost,Senate confirms Trump lawyer Emil Bove for fed...,https://nypost.com/2025/07/29/us-news/senate-c...,122
11,2025-07-30-14-30-00 +0000,wsj,The Senate confirmed former Trump defense atto...,https://www.wsj.com/politics/policy/trumps-for...,117
218,2025-07-29-21-42-41 +0000,nypost,Top Zelensky official praises Trump for giving...,https://nypost.com/2025/07/29/us-news/top-zele...,113
298,2025-07-29-17-49-06 +0000,nypost,"Moscow taunts Trump by comparing him to Biden,...",https://nypost.com/2025/07/29/world-news/mosco...,113


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
209,126,2025-07-29-22-00-59 +0000,nyt,"Trump Says Gaza Will Get More Aid, but Israel ...",https://www.nytimes.com/2025/07/29/us/politics...
8,73,2025-07-30-14-33-42 +0000,wapo,"Tsunami live updates: Waves reach Hawaii, Cali...",https://www.washingtonpost.com/world/2025/07/2...
233,68,2025-07-29-21-18-23 +0000,wapo,New details emerge in New York City shooting i...,https://www.washingtonpost.com/nation/2025/07/...
154,60,2025-07-30-01-39-32 +0000,nypost,Senate confirms Trump lawyer Emil Bove for fed...,https://nypost.com/2025/07/29/us-news/senate-c...
279,57,2025-07-29-18-32-06 +0000,nypost,UK will recognize Palestinian state unless Isr...,https://nypost.com/2025/07/29/world-news/uk-wi...
105,53,2025-07-30-09-30-00 +0000,wsj,The shake-up for charitable giving inside the ...,https://www.wsj.com/politics/policy/trump-tax-...
21,48,2025-07-30-14-16-00 +0000,wsj,Any further Bank of Canada rate reduction depe...,https://www.wsj.com/articles/bank-of-canada-st...
59,46,2025-07-30-12-43-30 +0000,nypost,Here’s what will happen to Shane Tamura’s brai...,https://nypost.com/2025/07/30/us-news/nyc-gunm...
226,43,2025-07-29-21-30-24 +0000,nypost,Ukrainian mom-to-be expecting first child afte...,https://nypost.com/2025/07/29/world-news/ukrai...
169,36,2025-07-30-00-37-54 +0000,bbc,"No deal on China tariffs until Trump agrees, s...",https://www.bbc.com/news/articles/cdxyv2y48nzo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
